In [ ]:
# python -m llava.serve.controller --host 0.0.0.0 --port 10000


# python -m llava.serve.model_worker --host 0.0.0.0 --controller http://localhost:10000 --port 40000 --worker http://localhost:40000 --model-path /rmt/quail/captioner-1.4-llava-v1.6-llama3.1-CLIP-8b-ft-1


# python -m llava.serve.model_worker --host 0.0.0.0 --controller http://localhost:10000 --port 40000 --worker http://localhost:40000 --model-path /rmt/quail/captioner-1.3-llava-v1.6-llama3-8b-ft-1

# python -m llava.serve.gradio_web_server --controller http://localhost:10000 --model-list-mode reload --share

2024-08-27 17:51:18,997 [INFO] UniLogger: UniLoader.loads: .json LOADED from "/rmt/yada/dev/tr-core/notebooks/vpred_api_default_workflow_api.json" in 0.00s


In [ ]:
from typing import Dict, Any

class WorkflowModifier:
    def _create_tag_mapping(self, api_workflow: Dict[str, Any]) -> Dict[str, str]:
        """Creates a mapping between tags and their corresponding node IDs."""
        tag_mapping = {}
        for node_id, node_data in api_workflow.items():
            if node_data['class_type'] == 'TaggedAny':
                tag = node_data['inputs'].get('tag')
                if tag:
                    tag_mapping[tag] = node_id
        return tag_mapping

    def get_default_values(self, api_workflow: Dict[str, Any]) -> Dict[str, Any]:
        """Returns a dictionary of modifiable values in the overrides format."""
        tag_mapping = self._create_tag_mapping(api_workflow)
        modifiable_values = {}
        for tag, node_id in tag_mapping.items():
            content = api_workflow[node_id]['inputs'].get('content')
            if content:
                modifiable_values[tag] = content
        return modifiable_values

    def modify_workflow(self, api_workflow: Dict[str, Any], overrides: Dict[str, Any]) -> Dict[str, Any]:
        """Modifies the workflow using the given overrides."""
        tag_mapping = self._create_tag_mapping(api_workflow)
        for key, value in overrides.items():
            if key in tag_mapping:
                node_id = tag_mapping[key]
                # Update the 'content' field in the corresponding node
                api_workflow[node_id]['inputs']['content'] = str(value).strip()
        return api_workflow

    def __call__(self, api_workflow: Dict[str, Any], overrides: Dict[str, Any]) -> Dict[str, Any]:
        """Allows the class instance to be called directly to modify the workflow."""
        return self.modify_workflow(api_workflow, overrides)



import unibox as ub

wf = ub.loads("/rmt/yada/dev/tr-core/notebooks/vpred_api_default_workflow_api.json")
        

overrides = {
    'api_steps': 28,
    'api_positive': '1girl, long hair, black hair'
}


modifier = WorkflowModifier()

# Example usage
res = modifier(wf, overrides)
res

In [8]:
from training_flow.utils.run_comfy_api import WorkflowExecutor

executor = WorkflowExecutor()
res = executor.run_workflow(res)

In [27]:
from typing import Dict, Any, Iterator, Tuple

class Workflow:
    def __init__(self, raw_json: Dict[str, Any]):
        self.raw_json = raw_json
        self._modifiable_keys = self._extract_modifiable_keys()

    def _extract_modifiable_keys(self) -> Dict[str, Tuple[str, Any]]:
        """Extracts and returns a dictionary of modifiable keys and their content."""
        modifiable_keys = {}
        for node_id, node_data in self.raw_json.items():
            if node_data['class_type'] == 'TaggedAny':
                tag = node_data['inputs'].get('tag')
                content = node_data['inputs'].get('content')
                if tag and content:
                    modifiable_keys[tag] = (node_id, content)
        return modifiable_keys

    def get_modifiable_keys(self) -> Dict[str, Any]:
        """Returns a dictionary of modifiable keys and their content."""
        return {key: value[1] for key, value in self._modifiable_keys.items()}

    def update_modifiable_keys(self, overrides: Dict[str, Any]) -> None:
        """Updates the modifiable keys using the given overrides."""
        for key, value in overrides.items():
            if key in self._modifiable_keys:
                node_id, _ = self._modifiable_keys[key]
                self.raw_json[node_id]['inputs']['content'] = value
                self._modifiable_keys[key] = (node_id, value)

    def export(self) -> Dict[str, Any]:
        """Exports the workflow as a dictionary."""
        return self.raw_json

    def __getitem__(self, key: str) -> Any:
        """Allows direct access to raw_json like a dictionary."""
        return self.raw_json[key]

    def __setitem__(self, key: str, value: Any) -> None:
        """Allows direct setting of items in raw_json like a dictionary."""
        self.raw_json[key] = value
        self._modifiable_keys = self._extract_modifiable_keys()  # Refresh modifiable keys

    def __delitem__(self, key: str) -> None:
        """Allows direct deletion of items in raw_json like a dictionary."""
        del self.raw_json[key]
        self._modifiable_keys = self._extract_modifiable_keys()  # Refresh modifiable keys

    def __iter__(self) -> Iterator[Tuple[str, Any]]:
        """Allows iteration over the raw_json as key-value pairs."""
        return iter(self.raw_json.items())

    def __len__(self) -> int:
        """Returns the length of the raw_json."""
        return len(self.raw_json)

    def __repr__(self) -> str:
        """Custom representation for the Workflow class."""
        return f"<Workflow with {len(self.raw_json)} nodes> | modifiable keys: {list(self._modifiable_keys.keys())}"

In [28]:
workflow = Workflow(wf)

# Access the workflow like a normal dictionary
print(workflow['3'])  # Accessing a node

{'inputs': {'seed': 764222903364357, 'steps': ['29', 0], 'cfg': ['22', 0], 'sampler_name': ['23', 0], 'scheduler': 'sgm_uniform', 'denoise': 1, 'model': ['26', 0], 'positive': ['27', 0], 'negative': ['28', 0], 'latent_image': ['5', 0]}, 'class_type': 'KSampler', '_meta': {'title': 'KSampler'}}


In [29]:
# Special handling of modifiable keys
modifiable_keys = workflow.get_modifiable_keys()
modifiable_keys

{'api_batchsize': '2',
 'api_positive': 'new positive content',
 'api_negative': 'new negative content',
 'api_width': '896\n',
 'api_height': '1152',
 'api_cfg': '0.9',
 'api_sampler_name': 'euler_cfg_pp',
 'api_steps': 28}

In [30]:
# Update the modifiable keys
overrides = {'api_batchsize': '2',
 'api_positive': 'new positive content',
 'api_negative': 'new negative content',
 'api_width': '896',
 'api_height': '1152',
 'api_cfg': '0.9',
 'api_sampler_name': 'euler_cfg_pp',
 'api_steps': 28
 }


workflow.update_modifiable_keys(overrides)
# Export the modified workflow
exported_workflow = dict(workflow)  # or workflow.export()
print(exported_workflow)

{'3': {'inputs': {'seed': 764222903364357, 'steps': ['29', 0], 'cfg': ['22', 0], 'sampler_name': ['23', 0], 'scheduler': 'sgm_uniform', 'denoise': 1, 'model': ['26', 0], 'positive': ['27', 0], 'negative': ['28', 0], 'latent_image': ['5', 0]}, 'class_type': 'KSampler', '_meta': {'title': 'KSampler'}}, '4': {'inputs': {'ckpt_name': 'yada_checkpoints/qft_v5c-c53_v5c-logfav-9.6_60k_cont2/checkpoint-e35_s10000.safetensors'}, 'class_type': 'CheckpointLoaderSimple', '_meta': {'title': 'Load Checkpoint'}}, '5': {'inputs': {'width': ['18', 0], 'height': ['19', 0], 'batch_size': ['10', 0]}, 'class_type': 'EmptyLatentImage', '_meta': {'title': 'Empty Latent Image'}}, '8': {'inputs': {'samples': ['3', 0], 'vae': ['4', 2]}, 'class_type': 'VAEDecode', '_meta': {'title': 'VAE Decode'}}, '9': {'inputs': {'filename_prefix': 'ComfyUI', 'images': ['8', 0]}, 'class_type': 'SaveImage', '_meta': {'title': 'Save Image'}}, '10': {'inputs': {'tag': 'api_batchsize', 'enforce_type': 'auto', 'content': '2'}, 'cla

In [32]:
workflow

<Workflow with 18 nodes> | modifiable keys: ['api_batchsize', 'api_positive', 'api_negative', 'api_width', 'api_height', 'api_cfg', 'api_sampler_name', 'api_steps']

In [20]:
from training_flow.utils.run_comfy_api import WorkflowExecutor

executor = WorkflowExecutor()
res = executor.run_workflow(res)

In [ ]:
res